# 使用自动化机器学习

有多种可供你用于训练模型的机器学习算法，有时很难确定针对特定数据和预测需求的最有效算法。此外，通过使用标准化、缺失特征填补等技术对训练数据进行预处理，可以显著影响模型的预测性能。在寻找满足您的要求的*最佳*模型时，可能需要尝试多种算法和预处理转换的组合；这需要大量的时间和计算资源。

通过 Azure 机器学习，可以使用不同的算法和预处理选项来自动比较已训练的模型。可以使用 [Azure 机器学习工作室](https://ml/azure.com)中的可视界面或 SDK 来利用此功能。SDK 使你能够更好地控制自动化机器学习试验的设置，但可视界面更易于使用。在本实验中，你将使用 SDK 探索自动化机器学习。

## 连接到工作区

你首先需要使用 Azure ML SDK 连接到工作区。

> **注意**： 如果 Azure 订阅的身份验证会话在你完成上一练习后已过期，系统将提示你重新进行身份验证。

In [ ]:
import azureml.core
from azureml.core import Workspace

# 从保存的配置文件加载工作区
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 准备用于自动化机器学习的数据

无需创建训练脚本来进行自动化机器学习，但需要加载训练数据。在本例中，你将创建一个包含糖尿病患者详细信息的数据集（与之前实验中的操作相同），然后将此数据集拆分为两个数据集：一个用于训练，另一个用于模型验证。

In [ ]:
from azureml.core import Dataset

default_ds = ws.get_default_datastore()

if 'diabetes dataset' not in ws.datasets:
    default_ds.upload_files(files=['./data/diabetes.csv', './data/diabetes2.csv'], # Upload the diabetes csv files in /data
                        target_path='diabetes-data/', # Put it in a folder path in the datastore
                        overwrite=True, # Replace existing files of the same name
                        show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'diabetes-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='diabetes dataset',
                                description='diabetes data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')


# 将数据集拆分为训练子集和验证子集
diabetes_ds = ws.datasets.get("diabetes dataset")
train_ds, test_ds = diabetes_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

## 配置自动化机器学习

现在可以配置自动化机器学习试验。为此，你将需要一个运行配置，其中包括试验环境所需的包以及一组配置设置，这些设置将指定要尝试的组合数量、评估模型时要使用的指标等。

> **注意**： 在本例中，你将在本地计算上运行自动机器学习试验，以避免等待群集启动。这将导致每个迭代（子运行）以串行方式而不是并行方式运行。因此，我们将试验限制为 6 次迭代，以减少所需的时间。在现实中，你可能会在计算群集上尝试更多的迭代。

In [ ]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target='local',
                             enable_local_managed=True,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='Diabetic',
                             iterations=6,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=4,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")

## 运行自动化机器学习试验

现在可以开始操作了。现在运行自动化机器学习试验。

In [ ]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'diabetes_automl')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

## 确定性能最佳模型

完成试验后，在小部件中查看输出，然后单击生成最佳结果的运行以查看其详细信息。
然后单击链接以在 Azure 门户中查看试验详细信息并查看整体试验详细信息，然后再查看生成最佳结果的单个运行的详细信息。此处提供许多关于已生成模型的性能的信息。

现在获取最佳运行以及它生成的模型。

In [ ]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

自动化机器学习包括尝试预处理数据的选项，这是通过使用 [Scikit-Learn 转换管道](https://scikit-learn.org/stable/modules/compose.html#combining-estimators)（不要与 Azure 机器学习混淆！）完成的。所生成的模型包括在推理之前转换数据的步骤。可以在模型中查看步骤，如下所示：

In [ ]:
for step in fitted_model.named_steps:
    print(step)

最后，找到性能最佳模型后就可以进行注册。

In [ ]:
from azureml.core import Model

# 注册模型
best_run.register_model(model_path='outputs/model.pkl', model_name='diabetes_model_automl',
                        tags={'Training context':'Auto ML'},
                        properties={'AUC': best_run_metrics['AUC_weighted'], 'Accuracy': best_run_metrics['accuracy']})

# 列出已注册的模型
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

> **更多信息**： 有关自动化机器学习的详细信息，请参阅 [Azure ML 文档](https://docs.microsoft.com/azure/machine-learning/how-to-configure-auto-train)。